In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import numpy as np
import pandas as pd
import h5py as  h
import torch
import torch.nn as nn
import skimage.transform as skt
from tqdm import tqdm

In [3]:
class AttrDict(dict):
    def __init__(self, *args, **kwargs):
        super(AttrDict, self).__init__(*args, **kwargs)
        self.__dict__ = self
args = AttrDict({"num_trajs":807,
        "sl":96,
        "batch_size":20,
        "chan_size":3,
        "num_particles":30,
        "window_length":6,
        "h":64,
        "img_size":(132,96),
        "emb_obs":32,
        "emb_act":32,
        "map_emb":32,
        "dropout":0.1, #we changed this from 0.5
        "obs_num":5,
        "resamp_alpha":0.5,
        "bp_length":10, 
        "clip":1.0,
        "lr":0.0005,
        "bpdecay":0.1,
        "l2_weight":1.0,
        "l1_weight":0.0})

In [4]:
shapes = []
def test(d, f, i):
    hh = h.File(dataloc + folder + "/" + i,'r')['feature']
    exfile = skt.resize(hh, (3, 132, hh.shape[2]))
    shapes.append(exfile.shape)
    return exfile
dataloc = "data/DCASE_2019_feature/"
folders = [i for i in os.listdir(dataloc) if not "." in i]
data = {}
for folder in folders:
    dataset = {i:test(dataloc, folder, i) for i in os.listdir(dataloc + folder)}
    data[folder] = dataset


In [5]:
keys = list(data['gcc_mic_dev'].keys())
e = {key:{k:data[k][key] for k in data if key in data[k]} for key in keys}

In [6]:
labels_order = []
labels = []
with open("data/DCASE_2019_feature/DCSSE_2019_labels.txt", 'r') as lbl:
    for line in lbl.readlines():
        line = line.split()
        labels.append((int(line[2]),int(line[3])))
        labels_order.append(line[0])
labels = np.array(labels)

In [7]:
feat_order = ['gcc_mic_dev', 'logmel_mic_dev', 'intensity_foa_dev', 'logmel_foa_dev']

In [8]:
inputs = np.zeros((labels.shape[0], len(feat_order), args.chan_size, *args.img_size))
for lblidx, lbl in enumerate(labels_order):
    inputs[lblidx] = np.stack([e[lbl + ".h5"][k] for k in feat_order])

In [9]:
from custom_model import AudioDataset, AudioLocalizer
from torch.utils.data import DataLoader

In [10]:
ds = AudioDataset(inputs, labels)
train_dataset, test_dataset = torch.utils.data.random_split(ds, [(train_size := len(ds)*4//5), len(ds) - train_size],generator=torch.Generator().manual_seed(42))

In [11]:
train_loader = DataLoader(train_dataset, batch_size=20,
                            num_workers=8, pin_memory=True, shuffle=False, drop_last = True)
test_loader = DataLoader(test_dataset, batch_size=20,
                            num_workers=8, pin_memory=True, shuffle=False, drop_last = True)
model = AudioLocalizer(args)
optimizer = torch.optim.RMSprop(model.parameters(), lr=args.lr)

In [12]:
f = enumerate(train_loader)

In [16]:
model.train()

for iteration, data in f:

    gcc, mic, intensity, foa, lbl = data

    print(type(foa), foa.shape)


    model.zero_grad()
    loss, log_loss, particle_pred = model.step(
        gcc, mic, intensity, foa, lbl, args)
    loss.backward()
    if args.clip > 0:
        torch.nn.utils.clip_grad_norm_(model.parameters(), args.clip)
    optimizer.step()

<class 'torch.Tensor'> torch.Size([20, 3, 132, 96])
embedding.shape torch.Size([20, 1, 256])
embedding.shape torch.Size([600, 1, 256])


IndexError: too many indices for tensor of dimension 2